### Functionalizing
- Funtional API로 모델을 만들기
- pixel값 1 ~ 255를 0 ~ 1 사잇값 Float32로 만들기
- One Hot Encoding Label에 적용
- 학습과 검증 데이터로 나누기
- compile, 학습/예측/평가
- 시각화

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"


In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer, Input,Dense,Flatten 
from tensorflow.keras.models import Model
import tensorflow as tf

In [12]:
INPUT_SIZE=28

def create_model():

    input_tensor = Input(shape=(INPUT_SIZE,INPUT_SIZE))
    x = Flatten()(input_tensor)
    x= Dense(100,activation='relu')(x)
    x = Dense(30,activation='relu')(x)
    output=Dense(10,activation='softmax')(x)

    model = Model(inputs=input_tensor,outputs=output)

    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
Total params: 81,840
Trainable params: 81,840
Non-trainable params: 0
_________________________________________________________________


2021-11-15 22:53:20.435590: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-15 22:53:20.435768: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-15 22:53:20.435898: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-15 22:53:20.441382: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-11-15 22:53:20.441536: W tensorflow/stream_executor/platform/default/dso_loader

In [8]:
#0 ~ 1 사이값의 float32로 변경하는 함수

def get_preprocessed_data(images,labels):
    images = np.array(images/255.0,dtype=np.float32)
    labels = np.array(labels,dtype=np.float32)
    return images,labels 


# 0 ~ 1 사이값 float32로 변경하는 함수를 호출한 뒤 ohe 적용

def get_preprocessed_ohe(images,labels):
    images,labels = get_preprocessed_data(images,labels)
    oh_labels = to_categorical(labels)
    return images,labels

# 학습 / 검증 / 테스트 데이터세트에 전처리 및 OHE 적용한 후 반환

def get_train_valid_test_set(train_images,train_labels,test_images,test_labels,valid_size=0.15,random_state=2021):

    train_images,train_oh_labels=get_preprocessed_ohe(train_images,train_labels)
    test_images,test_oh_labels=get_preprocessed_ohe(test_images,test_labels)


    #검증 데이터

    tr_images,val_images,tr_oh_labels,val_oh_labels = train_test_split(train_images,train_oh_labels,test_size=valid_size,random_state=2021)
    return (tr_images,tr_oh_labels),(val_images,val_oh_labels),(test_images,test_oh_labels)

#### Fashion MNIST 데이터 재로딩 및 전처리 적용
- 학습 / 검증 / 데이터 셋 생성

In [27]:
from tensorflow.keras.datasets import fashion_mnist 

fmist = fashion_mnist.load_data()

(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()